<a href="https://colab.research.google.com/github/Harikaraja/Fake-News-Classifier-using-LSTM/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/MyDrive/Deep Learning projects/fake-news/train.csv"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(path)

In [ ]:
print(df.head())

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1  


In [ ]:
df = df.dropna()          # Drop all the Nan vlaues

In [ ]:
X = df.drop('label',axis=1)

In [ ]:
y = df['label']

In [ ]:
np.shape(X)

(18285, 4)

In [ ]:
np.shape(y)

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Embedding,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM

In [ ]:
voc_size=5000

**One Hot Representaion**

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(messages)

       index     id                                              title  \
0          0      0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1      1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2      2                  Why the Truth Might Get You Fired   
3          3      3  15 Civilians Killed In Single US Airstrike Hav...   
4          4      4  Iranian woman jailed for fictional unpublished...   
...      ...    ...                                                ...   
18280  20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
18281  20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
18282  20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
18283  20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
18284  20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1      

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    #print(review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    '''' '.join(review): This line joins the stemmed words in the review list back into a single string, separated by spaces. This step is necessary to convert the list of words back into a text sentence.'''
    #print(review)
    corpus.append(review)

In [ ]:
print(corpus)

['hous dem aid even see comey letter jason chaffetz tweet', 'flynn hillari clinton big woman campu breitbart', 'truth might get fire', 'civilian kill singl us airstrik identifi', 'iranian woman jail fiction unpublish stori woman stone death adulteri', 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart', 'beno hamon win french socialist parti presidenti nomin new york time', 'back channel plan ukrain russia courtesi trump associ new york time', 'obama organ action partner soro link indivis disrupt trump agenda', 'bbc comedi sketch real housew isi caus outrag', 'russian research discov secret nazi militari base treasur hunter arctic photo', 'us offici see link trump russia', 'ye paid govern troll social media blog forum websit', 'major leagu soccer argentin find home success new york time', 'well fargo chief abruptli step new york time', 'anonym donor pay million releas everyon arrest dakota access pipelin', 'fbi close hillari', 'chuck todd

In [ ]:
onehot_representation = [one_hot(sent,voc_size) for sent in corpus]
print(onehot_representation)

[[1958, 1643, 3801, 2200, 2403, 4950, 1969, 2904, 3263, 3835], [3424, 1850, 1904, 3472, 2762, 2456, 4316], [2279, 1620, 3074, 4307], [3773, 2795, 773, 2326, 3921, 770], [2372, 2762, 1329, 3302, 1508, 3916, 2762, 1275, 1130, 2198], [2512, 4939, 3448, 2840, 4558, 1004, 106, 3073, 4653, 3905, 3661, 4328, 4295, 2482, 4316], [3614, 3745, 2781, 4206, 3428, 3861, 4132, 333, 3383, 4933, 586], [3158, 532, 1284, 1290, 759, 779, 1004, 63, 3383, 4933, 586], [2518, 3729, 4466, 3395, 3755, 4730, 2451, 2539, 1004, 3713], [3961, 537, 817, 3979, 4476, 1248, 1086, 1969], [3254, 1674, 4595, 3566, 678, 439, 4935, 3911, 1046, 3687, 946], [2326, 2756, 2403, 4730, 1004, 759], [3435, 1683, 3693, 1342, 3714, 2073, 4714, 586, 2655], [640, 2655, 565, 2750, 1332, 4633, 50, 3383, 4933, 586], [4944, 4009, 3350, 1956, 3693, 3383, 4933, 586], [3103, 1909, 2627, 2047, 3137, 2887, 1113, 3228, 3754, 370], [1729, 609, 1850], [4076, 4084, 3962, 469, 1004, 3986, 2107, 4316], [2629, 232, 1904, 1706, 3683, 2019, 3932, 2612, 

In [ ]:
sent_length = 20
embedded_sent = pad_sequences(onehot_representation, padding='pre',maxlen=sent_length)
print(embedded_sent)

[[   0    0    0 ... 2904 3263 3835]
 [   0    0    0 ... 2762 2456 4316]
 [   0    0    0 ... 1620 3074 4307]
 ...
 [   0    0    0 ... 3383 4933  586]
 [   0    0    0 ... 4530  219 4146]
 [   0    0    0 ...  284  213 2142]]


In [ ]:
dim = 40
model=Sequential()
model.add(Embedding(voc_size,dim,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_sent)
y_final=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 53ms/step - loss: 0.3367 - accuracy: 0.8446 - val_loss: 0.1893 - val_accuracy: 0.9190
Epoch 2/10
192/192 [==============================] - 9s 46ms/step - loss: 0.1421 - accuracy: 0.9457 - val_loss: 0.1916 - val_accuracy: 0.9236
Epoch 3/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0967 - accuracy: 0.9645 - val_loss: 0.2244 - val_accuracy: 0.9087
Epoch 4/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0674 - accuracy: 0.9771 - val_loss: 0.2767 - val_accuracy: 0.9206
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0432 - accuracy: 0.9857 - val_loss: 0.2594 - val_accuracy: 0.9152
Epoch 6/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0312 - accuracy: 0.9903 - val_loss: 0.3730 - val_accuracy: 0.9183
Epoch 7/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0177 - accuracy: 0.9953 - val_loss: 0.3614 - val_accuracy: 0.9

In [ ]:
y_pred=model.predict(X_test)

189/189 [==============================] - 2s 9ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(y_pred)

[[9.99989927e-01]
 [1.45793765e-05]
 [2.81978402e-07]
 ...
 [7.62126525e-04]
 [9.97529805e-01]
 [9.99982417e-01]]


In [ ]:
print(y_test)

[1 0 0 ... 0 1 1]


In [ ]:
confusion_matrix(y_test,y_pred.round())

array([[3133,  286],
       [ 213, 2403]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred.round())

0.9173156586578294